In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import warnings
from tqdm import tqdm
import gc
import shapely

def build_river_network_from_vaa(flowlines_data):
    G = nx.DiGraph()
    
    # Add nodes
    G.add_nodes_from(flowlines_data['hydroseq'].dropna().values)
    
    # Add edges where uphydroseq is not null
    edges = [(row['uphydroseq'], row['hydroseq']) 
            for _, row in flowlines_data.iterrows() 
            if pd.notna(row['uphydroseq'])]
    
    G.add_edges_from(edges)
    return G

def process_county(county, flowline_mapping, river_graph, all_counties):
    fips = county['FIPS']
    county_flowlines = [hydroseq for hydroseq, county_fips in flowline_mapping.items() 
                       if county_fips == fips]
    
    if not county_flowlines:
        return {'FIPS': fips, 'upstream_area': 0, 'upstream_population': 0}
    
    try:
        upstream_hydroseqs = set()
        for hydroseq in county_flowlines:
            if hydroseq in river_graph:
                predecessors = nx.ancestors(river_graph, hydroseq)
                upstream_hydroseqs.update(predecessors)
                upstream_hydroseqs.add(hydroseq)
        
        upstream_counties_fips = {flowline_mapping.get(seq) 
                                for seq in upstream_hydroseqs 
                                if seq in flowline_mapping}
        upstream_counties_fips.discard(None)
        all_counties.geometry = all_counties.buffer(0)
        upstream_counties = all_counties[all_counties['FIPS'].isin(upstream_counties_fips)]
        total_area = upstream_counties.geometry.area.sum()
        total_population = upstream_counties['POPULATION'].sum()
             
        return {
            'FIPS': fips,
            'upstream_area': total_area,
            'upstream_population': total_population
        }
    except Exception as e:
        print(f"Error processing county {fips}: {e}")
        return {'FIPS': fips, 'upstream_area': 0, 'upstream_population': 0}

# Load counties
counties = gpd.read_file('counties_population/dtl_cnty.shp', 
                           columns=['FIPS', 'geometry', 'POPULATION'])

# Load NetworkNHDFlowline with required columns
flowlines = gpd.read_file('NHDPlus_H_National_Release_1_GDB.gdb', 
                         layer='NetworkNHDFlowline', 
                         columns=['hydroseq', 'uphydroseq', 'geometry'], 
                         engine="pyogrio")

# Convert data types and handle NaN
flowlines[['hydroseq', 'uphydroseq']] = flowlines[['hydroseq', 'uphydroseq']].fillna(0).astype(float).astype(int)

# Build river network directly from flowlines
river_graph = build_river_network_from_vaa(flowlines)

# Convert flowlines to 2D and prepare for sjoin
flowlines.geometry = flowlines.geometry.map(lambda geom: 
    shapely.wkb.loads(shapely.wkb.dumps(geom, output_dimension=2)))

# Project to a suitable CRS
flowlines = flowlines.to_crs('EPSG:26913')
counties = counties.to_crs('EPSG:26913')
counties['FIPS'] = counties['FIPS'].astype(str).str.zfill(5)

# Create the mapping between flowlines and counties
joined = gpd.sjoin(flowlines[['hydroseq', 'uphydroseq', 'geometry']], 
                  counties,
                  how='left', 
                  predicate='intersects')

mapping = joined.set_index('hydroseq')['FIPS'].dropna().to_dict()

# Process counties
results_list = []

for idx, county in tqdm(counties.iterrows(), total=len(counties)):
    result = process_county(county, mapping, river_graph, counties)
    results_list.append(result)
    if idx % 100 == 0:
        gc.collect()

results = pd.DataFrame(results_list)
counties = counties.merge(results, on='FIPS', how='left')
counties.to_file('counties_with_upstream_data.gpkg')